In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import itertools
import os
import warnings
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import sklearn as skl
import scipy.stats as spst
import sklearn.metrics as skmr
import xgboost as xgb
from tabpfn import TabPFNClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state

In [ ]:
Ds = list(range(3, 14))
method_infos = [
    ("XGBoost", xgb.XGBClassifier()),
    ("TabPFN-v2 (1)", TabPFNClassifier(device='cpu', n_estimators=1)),
    ("TabPFN-v2 (4)", TabPFNClassifier(device='cpu', n_estimators=4)),
    ("TabPFN-v2 (16)", TabPFNClassifier(device='cpu', n_estimators=16)),
]

In [ ]:
results_dict = {mname : {} for mname, _ in method_infos} 
pvalues_dict = {mname : {} for mname, _ in method_infos} 
for D in Ds:
    X = np.array(list(itertools.product([False, True], repeat=D))).astype(int)
    N = X.shape[0]
    y = np.sum(X, axis=1) % 2
    for mname, method in method_infos:
        y_test_agg = []
        y_test_pred_agg = []
        for n in range(N):
            X_train = np.r_[X[0:n, :], X[n+1:, :]]
            y_train = np.r_[y[0:n], y[n+1:]]
            X_test = X[n:n+1, :]
            y_test = y[n:n+1]
            method.fit(X_train, y_train)
            y_test_pred = method.predict(X_test)
            y_test_agg.append(y_test)
            y_test_pred_agg.append(y_test_pred)
        y_test_agg = np.concatenate(y_test_agg)
        y_test_pred_agg = np.concatenate(y_test_pred_agg)
        is_correct = (y_test_agg == y_test_pred_agg)
        acc = np.mean(is_correct)
        print(D, mname, acc)
        results_dict[mname][D] = acc
        pvalues_dict[mname][D] = spst.binomtest(k=np.sum(is_correct), n=N, p=0.5, alternative='greater').pvalue

In [ ]:
display(pd.DataFrame(results_dict))

ax = pd.DataFrame(results_dict).plot(marker='o')
ax.set_xticks(Ds)
#pd.DataFrame(results_dict).plot.bar(rot=0, ax=ax)
plt.savefig('parity-Nminus1.pdf')
plt.grid(True)

In [ ]:
halfresults_dict = {mname : {} for mname, _ in method_infos} 
for D in Ds:
    X = np.array(list(itertools.product([False, True], repeat=D))).astype(int)
    N = X.shape[0]
    y = np.sum(X, axis=1) % 2
    for mname, method in method_infos:
        y_test_agg = []
        y_test_pred_agg = []
        for n in range(N):
            X_train = np.r_[X[0:n, :], X[n+1:, :]]
            y_train = np.r_[y[0:n], y[n+1:]]
            rng = check_random_state(n)
            train_ixs = rng.choice(N - 1, size=N // 2, replace=False)
            X_train = X_train[train_ixs, :]
            y_train = y_train[train_ixs]
            X_test = X[n:n+1, :]
            y_test = y[n:n+1]
            method.fit(X_train, y_train)
            y_test_pred = method.predict(X_test)
            y_test_agg.append(y_test)
            y_test_pred_agg.append(y_test_pred)
        y_test_agg = np.concatenate(y_test_agg)
        y_test_pred_agg = np.concatenate(y_test_pred_agg)
        acc = np.mean(y_test_agg == y_test_pred_agg)
        print(D, mname, acc)
        halfresults_dict[mname][D] = acc

In [ ]:
ax = pd.DataFrame(halfresults_dict).plot(marker='o', alpha=0.2)
ax.set_xticks(Ds)
plt.savefig('parity-half.pdf')
plt.grid(True)

In [ ]:
quarterresults_dict = {mname : {} for mname, _ in method_infos} 
for D in [7]:
    X = np.array(list(itertools.product([False, True], repeat=D))).astype(int)
    N = X.shape[0]
    y = np.sum(X, axis=1) % 2
    for mname, method in method_infos:
        y_test_agg = []
        y_test_pred_agg = []
        for n in range(N):
            X_train = np.r_[X[0:n, :], X[n+1:, :]]
            y_train = np.r_[y[0:n], y[n+1:]]
            rng = check_random_state(n)
            train_ixs = rng.choice(N - 1, size=N // 4, replace=False)
            X_train = X_train[train_ixs, :]
            y_train = y_train[train_ixs]
            X_test = X[n:n+1, :]
            y_test = y[n:n+1]
            method.fit(X_train, y_train)
            y_test_pred = method.predict(X_test)
            y_test_agg.append(y_test)
            y_test_pred_agg.append(y_test_pred)
        y_test_agg = np.concatenate(y_test_agg)
        y_test_pred_agg = np.concatenate(y_test_pred_agg)
        acc = np.mean(y_test_agg == y_test_pred_agg)
        print(D, mname, acc)
        quarterresults_dict[mname][D] = acc

In [ ]:
ax = pd.DataFrame(quarterresults_dict).plot(marker='o')
ax.set_xticks(Ds)
plt.savefig('parity-quarter.pdf')
plt.grid(True)

In [ ]:
constantresults_dict = {mname : {} for mname, _ in method_infos} 
for D in list(range(7, 14)):
    X = np.array(list(itertools.product([False, True], repeat=D))).astype(int)
    N = X.shape[0]
    y = np.sum(X, axis=1) % 2
    for mname, method in method_infos:
        y_test_agg = []
        y_test_pred_agg = []
        for n in range(N):
            X_train = np.r_[X[0:n, :], X[n+1:, :]]
            y_train = np.r_[y[0:n], y[n+1:]]
            rng = check_random_state(n)
            train_ixs = rng.choice(N - 1, size=2 ** 7 - 1, replace=False)
            X_train = X_train[train_ixs, :]
            y_train = y_train[train_ixs]
            X_test = X[n:n+1, :]
            y_test = y[n:n+1]
            method.fit(X_train, y_train)
            y_test_pred = method.predict(X_test)
            y_test_agg.append(y_test)
            y_test_pred_agg.append(y_test_pred)
        y_test_agg = np.concatenate(y_test_agg)
        y_test_pred_agg = np.concatenate(y_test_pred_agg)
        acc = np.mean(y_test_agg == y_test_pred_agg)
        print(D, mname, acc)
        constantresults_dict[mname][D] = acc

In [ ]:
ax = pd.DataFrame(constantresults_dict).plot(marker='o')
ax.set_xticks(Ds)
plt.savefig('parity-constant.pdf')
plt.grid(True)